This file is for gathering solvent accessable surface areas. Unfortunately, I cannot run this with my main pipeline because it requires a newer version of python and biopython than I have in my main pipeline and I don't want to break my main pipeline.

In [1]:
from Bio.PDB.MMCIFParser import MMCIFParser
from Bio.PDB.SASA import ShrakeRupley
import pandas as pd
import os

In [2]:
# # Gather the solvent accessibility values for each protein

def get_sasa(row, path):
    
    fn = row['af_filename']
    fp = os.path.join(path, fn)

    p = MMCIFParser(QUIET=1)
    struct = p.get_structure('x', fp)
    sr = ShrakeRupley()
    sr.compute(struct, level='S')
    row['sasa'] = round(struct.sasa, 2)

    return row


In [3]:
path1 = './project_pipeline/data/input/Alphafold_cif'
path2 = './project_pipeline/data/input/Alphafold_multi_domain'

df = pd.read_csv('./project_pipeline/data/proteins.tsv', sep='\t')
df = df[['uniprot', 'af_filename']].drop_duplicates().reset_index(drop=True)
df = df.apply(get_sasa, args=(path1,), axis=1)

md = pd.read_csv('./project_pipeline/data/md_rmsds.tsv', sep='\t')
md = md[['uniprot', 'af_filename']].drop_duplicates().reset_index(drop=True)
md = md.apply(get_sasa, args=(path2,), axis=1)

In [4]:
df.to_csv('./project_pipeline/data/ai_full_depth_sasa.tsv', sep='\t', index=False)
md.to_csv('./project_pipeline/data/md_full_depth_sasa.tsv', sep='\t', index=False)